In [ ]:
import os, math, numpy as np, pandas as pd, tensorflow as tf
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

print("TF version:", tf.__version__)
print("Mounted input dirs:", os.listdir("/kaggle/input"))

DATA_DIR = "/kaggle/input/saig-tech-mastery-2025-art-style-classification/competition"

TRAIN_CSV = f"{DATA_DIR}/train.csv"
SUB_CSV   = f"{DATA_DIR}/sample_submission.csv"
IMAGE_DIR = f"{DATA_DIR}/image"

CLASS_LIST = ['Ink scenery','comic','cyberpunk','futuristic UI','lowpoly',
              'oil painting','pixel','realistic','steampunk','water color','UNK']
NUM_CLASSES = len(CLASS_LIST)
UNK_IDX     = CLASS_LIST.index("UNK")

IMG_SIZE = 256
BATCH    = 32
SEED     = 1337
np.random.seed(SEED); tf.random.set_seed(SEED)
AUTOTUNE = tf.data.AUTOTUNE

USE_TTA            = True      
USE_UNK_THRESHOLD  = False     


In [ ]:
train_df = pd.read_csv(TRAIN_CSV)
sub_df   = pd.read_csv(SUB_CSV)

style2idx = {s:i for i,s in enumerate(CLASS_LIST)}
train_df["label"] = train_df["style"].map(style2idx)

def to_path(uuid):
    p = f"{IMAGE_DIR}/{uuid}.jpg"
    if tf.io.gfile.exists(p): return p
    p2 = f"{IMAGE_DIR}/{uuid}.png"
    return p2 if tf.io.gfile.exists(p2) else p

train_df["path"] = train_df["uuid"].apply(to_path)
sub_df["path"]   = sub_df["uuid"].apply(to_path)

train_df = train_df[train_df["label"] != UNK_IDX].reset_index(drop=True)

tr_paths, val_paths, tr_labels, val_labels = train_test_split(
    train_df["path"].values, train_df["label"].values,
    test_size=0.2, stratify=train_df["label"].values, random_state=SEED
)
test_paths = sub_df["path"].values

print("Train / Val / Test:", len(tr_paths), len(val_paths), len(test_paths))

def decode_resize(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE), antialias=True)
    img = tf.clip_by_value(img/255.0, 0.0, 1.0)
    return img

def load_with_label(path, label):
    return decode_resize(path), label

data_aug = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.10),
    layers.RandomZoom(0.10),
    layers.RandomContrast(0.10),
], name="data_aug")

def make_ds(paths, labels=None, training=False):
    if labels is None:
        ds = tf.data.Dataset.from_tensor_slices(paths)
        ds = ds.map(lambda p: decode_resize(p), num_parallel_calls=AUTOTUNE)
    else:
        ds = tf.data.Dataset.from_tensor_slices((paths, labels))
        ds = ds.map(load_with_label, num_parallel_calls=AUTOTUNE)
        if training:
            ds = ds.shuffle(8192, seed=SEED, reshuffle_each_iteration=True)
    return ds.batch(BATCH).prefetch(AUTOTUNE)

train_ds = make_ds(tr_paths, tr_labels, training=True)
val_ds   = make_ds(val_paths, val_labels, training=False)
test_ds  = make_ds(test_paths, labels=None, training=False)


In [ ]:
base = tf.keras.applications.EfficientNetV2B0(
    include_top=False, weights="imagenet", input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base.trainable = False

inp = layers.Input((IMG_SIZE, IMG_SIZE, 3))
x = data_aug(inp, training=True)
x = tf.keras.applications.efficientnet_v2.preprocess_input(x * 255.0)
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.35)(x)         
x = layers.Dense(256, activation="relu")(x)
out = layers.Dense(NUM_CLASSES, activation="softmax")(x)
model = tf.keras.Model(inp, out)

model.summary()


In [ ]:
cls_w = compute_class_weight(
    class_weight="balanced",
    classes=np.arange(NUM_CLASSES-1),
    y=tr_labels[tr_labels < UNK_IDX]
)
class_weight = {i: float(cls_w[i]) for i in range(NUM_CLASSES-1)}

loss_stage1 = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss=loss_stage1, metrics=["accuracy"])

ckpt1 = ModelCheckpoint("best_stage1.keras", monitor="val_accuracy", mode="max",
                        save_best_only=True, verbose=1)
rlrop = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2,
                          min_lr=1e-6, verbose=1)
es1   = EarlyStopping(monitor="val_loss", patience=4,
                      restore_best_weights=True, verbose=1)

print("\n[Stage 1] Training...")
history1 = model.fit(
    train_ds, validation_data=val_ds,
    epochs=8, class_weight=class_weight,
    callbacks=[ckpt1, rlrop, es1], verbose=1
)


In [ ]:
base.trainable = True
for layer in base.layers[:150]:
    layer.trainable = False  

EPS = 0.05  
@tf.function
def sparse_cce_with_label_smoothing(y_true, y_pred):
    y_true = tf.cast(tf.reshape(y_true, [-1]), tf.int32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0)

    y_true_oh = tf.one_hot(y_true, depth=NUM_CLASSES, dtype=tf.float32)
    y_true_smooth = y_true_oh * (1.0 - EPS) + (EPS / tf.cast(NUM_CLASSES, tf.float32))

    per_example = -tf.reduce_sum(y_true_smooth * tf.math.log(y_pred), axis=-1)

    per_example = tf.ensure_shape(per_example, [None])
    return per_example

loss_stage2 = "sparse_categorical_crossentropy"
opt_ft = tf.keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4)

model.compile(optimizer=opt_ft, loss=loss_stage2, metrics=["accuracy"])

ckpt2 = tf.keras.callbacks.ModelCheckpoint(
    "best_stage2.keras", monitor="val_accuracy", mode="max",
    save_best_only=True, verbose=1
)
rlrop = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=2,
    min_lr=1e-6, verbose=1
)
es2 = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=4, restore_best_weights=True, verbose=1
)

print("\n[Stage 2] Fine-tuning (AdamW + more layers unfrozen)...")
history2 = model.fit(
    train_ds, validation_data=val_ds,
    epochs=6, class_weight=class_weight,
    callbacks=[ckpt2, rlrop, es2], verbose=1
)

model = tf.keras.models.load_model("best_stage2.keras", compile=False)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
print("[INFO] Loaded best Stage 2 (AdamW) model.")

In [ ]:
val_probs = model.predict(val_ds, verbose=0)
val_preds = val_probs.argmax(axis=1)
val_acc   = (val_preds == val_labels).mean()
print(f"[VAL] Accuracy: {val_acc:.4f}")


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

val_labels_set  = np.arange(NUM_CLASSES-1)           # 0..9
val_class_names = CLASS_LIST[:NUM_CLASSES-1]

cm = confusion_matrix(val_labels, val_preds, labels=val_labels_set)

plt.figure(figsize=(10,8))
try:
    import seaborn as sns
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=val_class_names, yticklabels=val_class_names)
except Exception:
    plt.imshow(cm, cmap="Blues"); plt.colorbar()
    plt.xticks(range(len(val_class_names)), val_class_names, rotation=45, ha="right")
    plt.yticks(range(len(val_class_names)), val_class_names)
plt.xlabel("Predicted"); plt.ylabel("True")
plt.title("Confusion Matrix on Validation (no UNK)")
plt.tight_layout(); plt.show()

print(classification_report(
    val_labels, val_preds,
    labels=val_labels_set, target_names=val_class_names, digits=4
))


In [ ]:
from sklearn.metrics import f1_score

val_probs = model.predict(val_ds, verbose=0)  
val_pred  = val_probs.argmax(1)
val_max   = val_probs.max(1)

best_tau, best_f1 = 0.0, -1
for tau in np.linspace(0.25, 0.75, 51):
    pred = np.where(val_max < tau, UNK_IDX, val_pred)
    f1 = f1_score(val_labels, pred, average="macro", labels=np.arange(NUM_CLASSES-1))
    if f1 > best_f1:
        best_tau, best_f1 = float(tau), float(f1)
print(f"[INFO] tau* = {best_tau:.3f}  (val macro-F1 = {best_f1:.4f})")

def tta_predict(model, ds):
    def gen_aug(k, flip):
        def _aug_batch(x):
            x = tf.image.rot90(x, k)
            return tf.image.flip_left_right(x) if flip else x
        return ds.map(lambda x: _aug_batch(x), num_parallel_calls=AUTOTUNE)
    aug_sets = [(k, f) for k in range(4) for f in (0,1)]
    probs = []
    for k, f in aug_sets:
        probs.append(model.predict(gen_aug(k, f), verbose=0))
    return np.mean(probs, axis=0)

print("\n[Inference] TTA-8 predicting test...")
test_probs = tta_predict(model, test_ds)
test_pred_idx = test_probs.argmax(axis=1)

test_max = test_probs.max(axis=1)
test_pred_idx = np.where(test_max < best_tau, UNK_IDX, test_pred_idx)

pred_labels = [CLASS_LIST[i] for i in test_pred_idx]
submission = pd.DataFrame({"uuid": sub_df["uuid"].values, "style": pred_labels})
out_path = "/kaggle/working/submission.csv"
submission.to_csv(out_path, index=False)
print(f"[OK] Wrote submission to: {out_path}")
submission.head()
